# A functional burst search and download service

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import time
from datetime import datetime

import asf_search as asf
import geopandas as gpd
import numpy as np
import pystac
import xarray as xr
from shapely import wkt

import bursts

In [2]:
def time_function_call(args,fun):
    start = time.time()
    result = fun(**args)
    end = time.time()
    time_s = end - start
    return result, time_s

In [3]:
safe_url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20220716T115019_20220716T115049_044125_054459_71A9.zip'

## Demoing interior objects
These objects will not likely not be accessed by users, but this function demonstrates how the metadata objects are created

In [4]:
auth = bursts.get_netrc_auth()

In [5]:
manifest, annotations = bursts.edl_download_metadata(safe_url, auth)

In [6]:
slc = bursts.SLCMetadata(safe_url, manifest, annotations)

In [7]:
swath = bursts.SwathMetadata(slc,'vv',swath_index=0)

In [8]:
burst = bursts.BurstMetadata(swath,burst_index=0)

In [9]:
item = burst.to_stac_item()
pystac.validation.validate(item)

['https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/item.json']

## Creating burst metadata datasets
Users have the option of creating either a geodatframe, or a STAC catalog. Burst data download is only enabled for the STAC catalog, but the geodataframe is great for in-notebook visualizations. Hopefully, users won't have to create these datasets themselves and will instead use an ASF-provided dataset.

In [10]:
start = datetime.strptime('20210101','%Y%m%d')
end = datetime.strptime('20210601','%Y%m%d')
mt_edgecumbe = 'POLYGON ((-135.7849 57.0417,-135.7243 57.0417,-135.7243 57.0654,-135.7849 57.0654,-135.7849 57.0417))'
insar_opts = dict(platform=[asf.PLATFORM.SENTINEL1], polarization=['VV','VV+VH','Dual VV'], beamMode='IW', processingLevel='SLC')
results = asf.geo_search(flightDirection='DESCENDING', intersectsWith=mt_edgecumbe, start=start, end=end, **insar_opts)
safe_urls = [x.properties['url'] for x in results]
len(safe_urls)

12

In [11]:
safe_urls

['https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210531T151557_20210531T151624_027150_033E3D_FC40.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210519T151556_20210519T151623_026975_03390A_7ABB.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210507T151556_20210507T151623_026800_033399_E754.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210425T151555_20210425T151622_026625_032E01_6189.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210413T151555_20210413T151622_026450_032864_0494.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210401T151554_20210401T151621_026275_0322CB_67B9.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210320T151554_20210320T151621_026100_031D4B_3AB5.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210224T151554_20210224T151621_025750_0311EE_7026.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210212T151554_202102

In [12]:
burst_list = bursts.get_burst_metadata(safe_urls, threads=10)

SUBMITTING | :   0%|          | 0/12 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/12 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/12 [00:00<?, ?it/s]

In [13]:
stac = bursts.generate_burst_stac_catalog(burst_list)
stac.describe()

* <Catalog id=burst-catalog>
    * <Collection id=372327_IW2>
      * <Item id=S1_SLC_20210531T151618_VV_372327_IW2>
      * <Item id=S1_SLC_20210519T151617_VV_372327_IW2>
      * <Item id=S1_SLC_20210507T151616_VV_372327_IW2>
      * <Item id=S1_SLC_20210425T151616_VV_372327_IW2>
      * <Item id=S1_SLC_20210413T151615_VV_372327_IW2>
      * <Item id=S1_SLC_20210401T151615_VV_372327_IW2>
      * <Item id=S1_SLC_20210320T151614_VV_372327_IW2>
      * <Item id=S1_SLC_20210224T151614_VV_372327_IW2>
      * <Item id=S1_SLC_20210212T151615_VV_372327_IW2>
      * <Item id=S1_SLC_20210131T151615_VV_372327_IW2>
      * <Item id=S1_SLC_20210119T151615_VV_372327_IW2>
      * <Item id=S1_SLC_20210107T151616_VV_372327_IW2>
    * <Collection id=372328_IW1>
      * <Item id=S1_SLC_20210531T151619_VV_372328_IW1>
      * <Item id=S1_SLC_20210519T151619_VV_372328_IW1>
      * <Item id=S1_SLC_20210507T151618_VV_372328_IW1>
      * <Item id=S1_SLC_20210425T151618_VV_372328_IW1>
      * <Item id=S1_SLC_2

## Explore with Geopandas

In [14]:
items = [x.to_dict() for x in stac.get_all_items()]
df = gpd.GeoDataFrame.from_features(items, crs="epsg:4326")
df.index = [x['id'] for x in items]

In [15]:
in_aoi = df.intersects(wkt.loads(mt_edgecumbe))
aoi = df.loc[in_aoi].copy()
print(f'Located {aoi.shape[0]} bursts that meet criteria')

Located 12 bursts that meet criteria


In [16]:
aoi.explore(style_kwds={'fill':None})

## View using STAC-browser

In [17]:
stac_path = bursts.save_stac_catalog_locally(stac)

In [18]:
bursts.initiate_stac_catalog_server(5555, stac_path.parent)

http://localhost:5555/catalog.json
 In stac-browser run:
 npm start -- --open --CATALOG_URL="http://localhost:5555/catalog.json" 


KeyboardInterrupt: 

## Download burst data
Using the STAC catalog, users can select an STAC item, then pass this item to `bursts.edl_download_burst_data`. The data is returned in a 2d numpy array with a complex float data type.

In [19]:
item_list = [stac.get_item(x,recursive=True) for x in aoi.index]

In [20]:
stack_dataset = bursts.edl_download_stack(item_list, threads=10)

SUBMITTING | :   0%|          | 0/12 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/12 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
stack_dataset

<xarray.Dataset>
Dimensions:                               (line: 1519, sample: 24229, time: 12)
Coordinates:
  * line                                  (line) int64 0 1 2 ... 1516 1517 1518
  * sample                                (sample) int64 0 1 2 ... 24227 24228
  * time                                  (time) datetime64[ns] 2021-05-31T15...
Data variables:
    S1_SLC_20210531T151613_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210519T151612_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210507T151612_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210425T151611_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210413T151611_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210401T151610_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210320T151610_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210224T151610_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210212T151610_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210131T151610_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210119T151611_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210107T151611_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j

In [22]:
stack_dataset.to_zarr('example.zarr')

TypeError: Invalid attribute in Dataset.attrs.

## xarray scrap

In [ ]:
np.save('./example_burst_data.npy', arrays[0])

In [ ]:
item = items[0]
array = np.load('./example_burst_data.npy')

In [ ]:
n_lines, n_samples = array.shape
dims = ('time','line','sample')
coords = ([item.datetime],list(range(n_lines)),list(range(n_samples)))
coords = {key:value for key,value in zip(dims,coords)}
burst_array = xr.DataArray(np.expand_dims(array,axis=0),coords=coords,dims=('time','line','sample'),attrs=item.properties)

In [ ]:
burst_array


In [ ]:
# aoi.explore(style_kwds={'fill':None})

## View using STAC-browser

In [ ]:
stac_path = bursts.save_stac_catalog_locally(stac)

In [ ]:
bursts.initiate_stac_catalog_server(5555,stac_path)

## Download burst data
Using the STAC catalog, users can select an STAC item, then pass this item to `bursts.edl_download_burst_data`. The data is returned in a 2d numpy array with a complex float data type.

In [ ]:
item_list = [stac.get_item(x,recursive=True) for x in aoi.index]

In [ ]:
stack_dataset = bursts.edl_download_stack(item_list, threads=10)

In [ ]:
stack_dataset

In [ ]:
# stack_dataset.to_zarr('example.zarr')

## xarray scrap

In [ ]:
np.save('./example_burst_data.npy',arrays[0])

In [ ]:
item = items[0]
array = np.load('./example_burst_data.npy')

In [ ]:
n_lines, n_samples = array.shape
dims = ('time','line','sample')
coords = ([item.datetime],list(range(n_lines)),list(range(n_samples)))
coords = {key:value for key,value in zip(dims,coords)}
burst_array = xr.DataArray(np.expand_dims(array,axis=0),coords=coords,dims=('time','line','sample'),attrs=item.properties)

In [ ]:
burst_array


In [ ]:
# aoi.explore(style_kwds={'fill':None})

## View using STAC-browser

In [ ]:
stac_path = bursts.save_stac_catalog_locally(stac)

In [ ]:
bursts.initiate_stac_catalog_server(5555,stac_path)

## Download burst data
Using the STAC catalog, users can select an STAC item, then pass this item to `bursts.edl_download_burst_data`. The data is returned in a 2d numpy array with a complex float data type.

In [ ]:
item_list = [stac.get_item(x,recursive=True) for x in aoi.index]

In [ ]:
stack_dataset = bursts.edl_download_stack(item_list, threads=10)

In [ ]:
stack_dataset

In [ ]:
# stack_dataset.to_zarr('example.zarr')

## xarray scrap

In [ ]:
np.save('./example_burst_data.npy',arrays[0])

In [ ]:
item = items[0]
array = np.load('./example_burst_data.npy')

In [ ]:
n_lines, n_samples = array.shape
dims = ('time','line','sample')
coords = ([item.datetime],list(range(n_lines)),list(range(n_samples)))
coords = {key:value for key,value in zip(dims,coords)}
burst_array = xr.DataArray(np.expand_dims(array,axis=0),coords=coords,dims=('time','line','sample'),attrs=item.properties)

In [ ]:
burst_array